In [2]:
import pandas as pd
import numpy as np
import openpyxl as el

In [3]:
input_file = './2024Apr1-2024Apr1CustomTransaction.csv'
output_file = './APRIL Amazon Manual report.xlsx'

In [4]:
data = pd.read_csv(input_file, header=7)

In [5]:
def convert_to_numeric(data, column_name):
    if data[column_name].dtype != np.float64 and data[column_name].dtype != np.int64:
        for i in range(len(data)):
            data[column_name][i] = float(data[column_name][i].replace(',', ''))
        data[column_name] = pd.to_numeric(data[column_name])
    return data

In [6]:
data = convert_to_numeric(data, 'quantity')
data = convert_to_numeric(data, 'product sales')
data = convert_to_numeric(data, 'product sales tax')
data = convert_to_numeric(data, 'total')

In [7]:
data.head()

,date/time,settlement id,type,order id,sku,description,quantity,marketplace,fulfilment,order city,...,gift wrap credits,giftwrap credits tax,promotional rebates,promotional rebates tax,marketplace withheld tax,selling fees,fba fees,other transaction fees,other,total
0,31 Mar 2024 23:33:31 UTC,21933240822,Order,204-8878064-1506743,WD22TB4NEW,DellDELL WD22TB4 Docking Thunderbolt Zwart (DE...,1,amazon.co.uk,Amazon,LIVINGSTON,...,0,0,-1.66,-0.33,0,-21.62,-3.90,0,0,151.97
1,1 Apr 2024 01:13:38 UTC,21933240822,Order,202-4967862-7875503,MY-0IHR-ICWL,"Dell D3100 Docking Station 3, 0 Ultra HD Tripl...",1,amazon.co.uk,Amazon,Colchester,...,0,0,-0.83,-0.17,0,-11.63,-3.90,0,0,60.46
2,1 Apr 2024 02:18:29 UTC,21933240822,Order,026-3582002-5081108,KE-YKQ1-JYS2,Dell P2219H 21.5 inch LED IPS Monitor - IPS Pa...,1,amazon.co.uk,Amazon,LONDON,...,0,0,0.00,0.00,0,-7.78,-10.24,0,0,90.96
3,1 Apr 2024 02:25:46 UTC,21933240822,Order,026-3582002-5081108,KE-YKQ1-JYS2,Dell P2219H 21.5 inch LED IPS Monitor - IPS Pa...,1,amazon.co.uk,Amazon,LONDON,...,0,0,0.00,0.00,0,-7.78,-10.24,0,0,90.96
4,1 Apr 2024 04:20:40 UTC,21933240822,Order,204-9276975-1855516,W6-TUC7-NPFN,"Dell (Genuine) Quick Release PC Monitor Stand,...",1,amazon.co.uk,Amazon,Doncaster,...,0,0,0.00,0.00,0,-4.05,-6.19,0,0,16.25


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date/time                 19 non-null     object 
 1   settlement id             19 non-null     int64  
 2   type                      19 non-null     object 
 3   order id                  19 non-null     object 
 4   sku                       19 non-null     object 
 5   description               19 non-null     object 
 6   quantity                  19 non-null     int64  
 7   marketplace               19 non-null     object 
 8   fulfilment                19 non-null     object 
 9   order city                19 non-null     object 
 10  order state               8 non-null      object 
 11  order postal              19 non-null     object 
 12  tax collection model      0 non-null      float64
 13  product sales             19 non-null     float64
 14  product sale

In [9]:
adjustment = data.loc[data['type'] == 'Adjustment']['total'].astype(np.float64).sum()

In [10]:
fba_inventory_fee = data.loc[data['type'] == 'FBA Inventory Fee']['total'].astype(np.float64).sum() * -1

In [11]:
service_fee = data.loc[data['type'] == 'Service Fee']['total'].astype(np.float64).sum() * -1

In [12]:
product_sales_tax = data.loc[data['type'] == 'Order']['product sales tax'].astype(np.float64).sum()

In [13]:
paid_transaction_gross = data.loc[data['type'] == 'Order']['product sales'].astype(np.float64).sum() + data.loc[data['type'] == 'Order']['product sales tax'].astype(np.float64).sum()

In [14]:
sales_after_amazon_sales_fees = data.loc[data['type'] == 'Order']['total'].astype(np.float64).sum()

In [15]:
amazon_fees_on_orders = paid_transaction_gross - sales_after_amazon_sales_fees

In [16]:
returns = data.loc[data['type'] == 'Refund']['total'].astype(np.float64).sum() * -1

In [17]:
reclaimed_vat = data.loc[data['type'] == 'Refund']['product sales tax'].astype(np.float64).sum() * -1

In [18]:
retrocharge = data.loc[data['type'] == 'Retrocharge']['total'].astype(np.float64).sum()

In [19]:
order_sku_quantity_tax_total = data.loc[data['type'] == 'Order'][['sku', 'quantity', 'product sales tax', 'total']]

In [20]:
new_order_sku_quantity_tax_total = order_sku_quantity_tax_total.groupby(['sku'], as_index=False).sum()

In [21]:
refund_sku_quantity = data.loc[data['type'] == 'Refund'][['sku', 'quantity']]

In [22]:
new_refund_sku_quantity = refund_sku_quantity.groupby(['sku'], as_index=False).sum()

In [23]:
order_refund_sku = np.concatenate((data.loc[data['type'] == 'Order']['sku'].unique(), data.loc[data['type'] == 'Refund']['sku'].unique()))

In [24]:
order_refund_sku = np.unique(order_refund_sku)

In [25]:
wb = el.load_workbook(output_file)

In [26]:
ws1 = wb['Sheet1']

In [27]:
ws1['B16'].value = adjustment

In [28]:
ws1['B9'].value = fba_inventory_fee

In [29]:
ws1['B8'].value = service_fee

In [30]:
ws1['B5'].value = product_sales_tax

In [31]:
ws1['B3'].value = paid_transaction_gross

In [32]:
ws1['B4'].value = sales_after_amazon_sales_fees

In [33]:
ws1['B6'].value = amazon_fees_on_orders

In [34]:
ws1['B13'].value = returns

In [35]:
ws1['B14'].value = reclaimed_vat

In [36]:
ws1['B21'].value = retrocharge

In [37]:
wb.save(output_file)

In [38]:
i = 27

while ws1.cell(row=i, column=1).value != None:
    if ws1.cell(row=i, column=1).value in new_order_sku_quantity_tax_total['sku'].unique():
        ws1.cell(row=i, column=5).value = new_order_sku_quantity_tax_total.loc[new_order_sku_quantity_tax_total['sku'] == ws1.cell(row=i, column=1).value]['quantity'].astype(np.float64).item()
    i = i + 1

In [39]:
wb.save(output_file)

In [40]:
i = 27

while ws1.cell(row=i, column=1).value != None:
    if ws1.cell(row=i, column=1).value in new_refund_sku_quantity['sku'].unique():
        ws1.cell(row=i, column=14).value = new_refund_sku_quantity.loc[new_refund_sku_quantity['sku'] == ws1.cell(row=i, column=1).value]['quantity'].astype(np.float64).item()
    i = i + 1

In [41]:
wb.save(output_file)

In [42]:
i = 27

while ws1.cell(row=i, column=1).value != None:
    if ws1.cell(row=i, column=1).value in new_refund_sku_quantity['sku'].unique():
        ws1.cell(row=i, column=15).value = ws1.cell(row=i, column=4).value * ws1.cell(row=i, column=14).value
    i = i + 1

In [43]:
wb.save(output_file)

In [44]:
i = 27

while ws1.cell(row=i, column=1).value != None:
    if ws1.cell(row=i, column=1).value in new_order_sku_quantity_tax_total['sku'].unique():
        ws1.cell(row=i, column=6).value = new_order_sku_quantity_tax_total.loc[new_order_sku_quantity_tax_total['sku'] == ws1.cell(row=i, column=1).value]['product sales tax'].astype(np.float64).item()
    i = i + 1

In [45]:
wb.save(output_file)

In [46]:
i = 27

while ws1.cell(row=i, column=1).value != None:
    if ws1.cell(row=i, column=1).value in new_order_sku_quantity_tax_total['sku'].unique():
        ws1.cell(row=i, column=7).value = new_order_sku_quantity_tax_total.loc[new_order_sku_quantity_tax_total['sku'] == ws1.cell(row=i, column=1).value]['total'].astype(np.float64).item()
    i = i + 1

In [47]:
wb.save(output_file)

In [48]:
report_file_sku = []

i = 27

while ws1.cell(row=i, column=1).value != None:
    report_file_sku.append(ws1.cell(row=i, column=1).value)
    i = i + 1

In [49]:
ws2 = wb['Sheet2']

In [50]:
for val in order_refund_sku:
    if val not in report_file_sku:
        i = 1
        while ws2.cell(row=i, column=1).value != None: 
            i = i + 1
        ws2.cell(row=i, column=1).value = val

In [51]:
wb.save(output_file)